## Setup and data importing

In [1]:
import pandas as pd
import numpy
import json
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import re
import pymc as pm
import pytensor.tensor as pt
import numpy as np

In [3]:
data_root = Path("data")
rows = []

for year_dir in data_root.iterdir():
    
    # gets rid of .DS_Store
    if year_dir.is_dir():
        year = year_dir.name
        
        # iterate through each year directory
        for json_path in year_dir.glob("*.json"):
            with open(json_path, "r") as f:
                obj = json.load(f)

            if isinstance(obj, list):
                records = obj
            else:
                records = [obj]

            for rec in records:
                abstract_raw = rec.get("awd_abstract_narration") or ""
                abstract_clean = abstract_raw.replace("\r\n", " ")
                rows.append(
                {
                    "year": year,
                    "file": json_path.name,
                    "awd_id": rec.get("awd_id"),
                    "awd_eff_date": rec.get("awd_eff_date"),
                    "awd_amount": rec.get("awd_amount"),
                    "org_dir_long_name": rec.get("org_dir_long_name"),
                    "abstract": abstract_clean
                }
            )

df = pd.DataFrame(rows)

In [4]:
terminated_awards = pd.read_csv("NSF-Terminated-Awards.csv")

In [5]:
df['awd_id'] = df['awd_id'].astype(str)
terminated_awards['Award ID'] = terminated_awards['Award ID'].astype(str)
df['terminated'] = df['awd_id'].isin(terminated_awards['Award ID']).astype(int)

## Clean, tokenize and vectorize the abstracts

In [6]:
def remove_numbers(text: str) -> str:
    if text is None:
        return ""
    text = str(text)
    return re.sub(r"\d+", " ", text)

tfidf = TfidfVectorizer(
    lowercase=True,
    preprocessor=remove_numbers,   
    stop_words="english",
    ngram_range=(1, 2),
    min_df=5,
    max_df=0.95,
)

X = tfidf.fit_transform(df["abstract"].fillna(""))

In [7]:
# make y our termination status
y = df["terminated"].astype(int).values

In [8]:
# reduce size of our X to make it less sparse using PCA/SVD
svd = TruncatedSVD(n_components=100)
X_small = svd.fit_transform(X)

In [10]:
# Inspect the svd components on both sides
terms = tfidf.get_feature_names_out()

def print_component_polar_terms(svd, terms, component_idx, top_n=10):
    component = svd.components_[component_idx]
    
    pos_idx = np.argsort(component)[-top_n:][::-1]  # largest (positive) weights
    neg_idx = np.argsort(component)[:top_n]          # most negative weights

    print(f"\nComponent {component_idx}")
    print("  Positive side:")
    for i in pos_idx:
        print(f"    {terms[i]:<25} {component[i]:.4f}")
    
    print("  Negative side:")
    for i in neg_idx:
        print(f"    {terms[i]:<25} {component[i]:.4f}")

# how many components fit?
n_components = svd.n_components

# inspect first 10 components 
for i in range(min(10, n_components)):
    print_component_polar_terms(svd, terms, i, top_n=10)


Component 0
  Positive side:
    The                       0.2228
    research                  0.1762
    project                   0.1758
    This                      0.1440
    students                  0.1339
    data                      0.1264
    new                       0.0950
    STEM                      0.0911
    science                   0.0846
    learning                  0.0832
  Negative side:
    events Knowledge          0.0000
    referred work             0.0000
    induction adolescent      0.0000
    role employees            0.0000
    induction parental        0.0000
    dissatisfaction turnover  0.0000
    home referred             0.0000
    Milkie children           0.0000
    maladjustment mediated    0.0000
    home domain               0.0000

Component 1
  Positive side:
    STEM                      0.3486
    GRFP                      0.2251
    students                  0.1465
    education                 0.1405
    program                   0.123

In [9]:
# Assume: X_small is dense numpy array (N x K)
#         y is 0/1 array of shape (N,)
with pm.Model() as logreg_model:

    # Priors
    β = pm.Normal("β", mu=0, sigma=1, shape=X_small.shape[1])
    α = pm.Normal("α", mu=0, sigma=2)

    # Linear predictor
    logits = α + pt.dot(X_small, β)

    # Likelihood
    y_obs = pm.Bernoulli("y_obs", logit_p=logits, observed=y)

    # Sample posterior
    trace = pm.sample(
        1000, tune=1000, target_accept=0.9, cores=4
    )

 Progress                    Draws   Divergences   Step size   Grad evals   Sampling Speed   Elapsed   Remaining  
 ───────────────────────────────────────────────────────────────────────────────────────────────────────────────── 
  ━━━━━━━━━━━━━━━━━━━━━━━━━   2000    0             0.219       15           1.58 draws/s     0:21:07   0:00:00    
  ━━━━━━━━━━━━━━━━━━━━━━━━━   2000    0             0.175       15           1.33 draws/s     0:25:01   0:00:00    
  ━━━━━━━━━━━━━━━━━━━━━━━━━   2000    0             0.237       15           1.44 draws/s     0:23:10   0:00:00    
  ━━━━━━━━━━━━━━━━━━━━━━━━━   2000    0             0.214       31           1.53 draws/s     0:21:46   0:00:00

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 1502 seconds.


In [12]:
import numpy as np

# Posterior mean of beta across chains and draws
beta_mean = trace.posterior["β"].mean(dim=("chain", "draw")).values  # shape (K,)

# Indices of the 5 largest coefficients by magnitude
top_k = 5
top_idx = np.argsort(np.abs(beta_mean))[-top_k:][::-1]

print("Top components by |beta|:")
for idx in top_idx:
    print(f"  Component {idx}: beta_mean = {beta_mean[idx]:.4f}")

Top components by |beta|:
  Component 10: beta_mean = 13.0528
  Component 3: beta_mean = -10.2517
  Component 1: beta_mean = 9.5272
  Component 13: beta_mean = 8.3566
  Component 30: beta_mean = -6.6413


In [13]:
for idx in top_idx:
    print(f"\n==== Component {idx} (beta_mean = {beta_mean[idx]:.4f}) ====")
    print_component_polar_terms(svd, terms, component_idx=idx, top_n=15)


==== Component 10 (beta_mean = 13.0528) ====

Component 10
  Positive side:
    quantum                   0.1932
    merit broader             0.1138
    award reflects            0.1138
    statutory mission         0.1138
    review criteria           0.1138
    statutory                 0.1138
    deemed worthy             0.1138
    NSF statutory             0.1138
    reflects NSF              0.1138
    mission deemed            0.1138
    worthy support            0.1138
    Foundation intellectual   0.1138
    using Foundation          0.1138
    impacts review            0.1138
    support evaluation        0.1138
  Negative side:
    data                      -0.1477
    ice                       -0.0854
    sponsoring scientist      -0.0849
    host institution          -0.0846
    sponsoring                -0.0813
    fellowship                -0.0792
    research                  -0.0759
    The                       -0.0754
    institution               -0.0717
    scien

### Categories with higher termination likelihood (positive β):

    1.	Awards with boilerplate abstract text (Component 10)
	2.	STEM education & outreach awards (Component 1)
	3.	Social science / policy / community research (Component 13 + 30)

### Categories with lower termination likelihood (negative β):

	1.	GRFP and fellowship-type awards (Component 3)
	2.	AI/robotics/cybersecurity workshop–style programs (Component 30)
	3.	Hard science / lab-heavy research (from negatives of components)